# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'11-06-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'11-06-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-11-07 05:24:52,33.93911,67.709953,41975,1554,34440,5981.0,Afghanistan,107.826326,3.702204
1,NaN,NaN,NaN,Albania,2020-11-07 05:24:52,41.15330,20.168300,23210,549,11861,10800.0,Albania,806.518869,2.365360
2,NaN,NaN,NaN,Algeria,2020-11-07 05:24:52,28.03390,1.659600,60800,2024,41510,17266.0,Algeria,138.651206,3.328947
3,NaN,NaN,NaN,Andorra,2020-11-07 05:24:52,42.50630,1.521800,5135,75,3858,1202.0,Andorra,6645.958714,1.460565
4,NaN,NaN,NaN,Angola,2020-11-07 05:24:52,-11.20270,17.873900,12223,300,5626,6297.0,Angola,37.190106,2.454389


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,41145,41268,41334,41425,41501,41633,41728,41814,41935,41975
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,20040,20315,20634,20875,21202,21523,21904,22300,22721,23210
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,57026,57332,57651,57942,58272,58574,58979,59527,60169,60800


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1529,1532,1533,1536,1536,1541,1544,1548,1554,1554
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,493,499,502,509,518,527,532,536,543,549
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1941,1949,1956,1964,1973,1980,1980,1999,2011,2024


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,34237,34239,34258,34321,34326,34342,34355,34362,34440,34440
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,10893,11007,11097,11189,11246,11367,11473,11578,11696,11861
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,39635,39635,40014,40201,40395,40577,40577,41001,41244,41510


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2103,2126,2141,2159,2173,2186,2197,2212,2230,2242
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,6743,6768,6888,6940,6966,6985,6995,7061,7097,7134
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1045,1055,1056,1060,1061,1065,1074,1079,1080,1090


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,31,31,31,31,31,31,31,31,31,30
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,69,69,71,71,71,71,74,77,77,83
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,9,9,9,9,9,9,9,9,9,9


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
634,1001.0,Autauga,Alabama,US,2020-11-07 05:24:52,32.539527,-86.644082,2242,30,0,2212.0,"Autauga, Alabama, US",4012.958886,1.338091
671,1075.0,Lamar,Alabama,US,2020-11-07 05:24:52,33.779950,-88.096680,544,7,0,537.0,"Lamar, Alabama, US",3940.601231,1.286765
672,1077.0,Lauderdale,Alabama,US,2020-11-07 05:24:52,34.901719,-87.656247,2874,45,0,2829.0,"Lauderdale, Alabama, US",3099.354032,1.565762


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,9733816,236073,3810791
India,8462080,125562,7819886
Brazil,5631181,162015,5118156


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,9733816,236073,3810791,5686952,2020-11-07 05:24:52,37.936303,-91.379001
India,8462080,125562,7819886,516632,2020-11-07 05:24:52,23.088275,81.806127
Brazil,5631181,162015,5118156,351010,2020-11-07 05:24:52,-12.669522,-48.480493
Russia,1720063,29654,1288096,402313,2020-11-07 05:24:52,54.546312,62.120860
France,1709716,39916,131810,1537990,2020-11-07 05:24:52,8.314863,-21.695174


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
Texas,977222,19046,0
California,963211,17931,0
Florida,832625,17014,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
Texas,977222,19046,0,958176,2020-11-07 05:24:52,31.660643,-98.653069
California,963211,17931,0,945280,2020-11-07 05:24:52,37.843962,-120.728594
Florida,832625,17014,0,815611,2020-11-07 05:24:52,28.940755,-82.700744
New York,522021,33664,0,488357,2020-11-07 05:24:52,42.544151,-75.474183
Illinois,465540,10397,0,455143,2020-11-07 05:24:52,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,317656,7157,0
Illinois,Cook,210266,5742,0
Florida,Miami-Dade,191838,3671,0
Texas,Harris,165967,2854,0
Arizona,Maricopa,164269,3682,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,317656,7157,0,310499,2020-11-07 05:24:52,34.308284,-118.228241,6037.0
Illinois,Cook,210266,5742,0,204524,2020-11-07 05:24:52,41.841448,-87.816588,17031.0
Florida,Miami-Dade,191838,3671,0,188167,2020-11-07 05:24:52,25.611236,-80.551706,12086.0
Texas,Harris,165967,2854,0,163113,2020-11-07 05:24:52,29.858649,-95.393395,48201.0
Arizona,Maricopa,164269,3682,0,160587,2020-11-07 05:24:52,33.348359,-112.491815,4013.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-04,41814,22300,59527,5045,11813,130,1205928,97150,27630,125099,...,1102305,3245,67779,93480,1203,56090,10,2063,16698,8427
2020-11-05,41935,22721,60169,5135,12102,130,1217028,99563,27644,132515,...,1126469,3309,68009,93921,1207,56672,10,2063,16770,8444
2020-11-06,41975,23210,60800,5135,12223,130,1228814,101773,27652,138979,...,1149791,3370,68139,94305,1212,57226,10,2067,16819,8471


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-04,1548,536,1999,75,296,3,32520,1448,907,1227,...,47832,61,577,814,35,504,1,601,349,248
2020-11-05,1554,543,2011,75,299,3,32766,1476,907,1268,...,48210,61,579,819,35,508,1,601,349,248
2020-11-06,1554,549,2024,75,300,3,33136,1506,907,1340,...,48565,61,579,821,35,511,1,602,349,250


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-04,34362,11578,41001,3734,5266,118,1017647,58811,25418,77036,...,2925,2770,65001,88326,1069,48224,8,1375,15819,7967
2020-11-05,34440,11696,41244,3858,5350,118,1030137,60135,25427,80604,...,2931,2814,65254,88701,1069,48680,8,1375,15827,7975
2020-11-06,34440,11861,41510,3858,5626,118,1042237,61782,25440,83707,...,2951,2853,65333,89099,1070,49537,8,1375,15862,7983


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2103,2126,2141,2159,2173,2186,2197,2212,2230,2242
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,516,523,525,530,534,535,533,540,541,544
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,2371,2405,2437,2465,2743,2765,2798,2820,2835,2874
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,877,885,899,916,941,946,958,968,977,983
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,6624,6649,6669,6702,6715,6736,6749,6776,6812,6851


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-04,197777,17860,0,250633,115812,950920,117637,75373,25534,49,...,269802,959811,121485,2267,1388,185836,111480,25988,244002,15044
2020-11-05,199158,18174,0,252768,117360,956854,121006,77060,25753,49,...,271771,969490,124292,2303,1388,187202,112550,26547,249924,15409
2020-11-06,200714,18699,0,254764,119230,963211,124469,78125,26035,49,...,273144,977222,127279,2326,1390,188770,114241,27087,256065,16405


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-04        2212    7061    1079  897   2188     659   1034    5001   
2020-11-05        2230    7097    1080  907   2222     661   1037    5039   
2020-11-06        2242    7134    1090  917   2253     662   1044    5077   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-04         1449      823  ...     765    186      888      197   
2020-11-05         1461      831  ...     788    190      899      200   
2020-11-06         1469      832  ...     900    207      957      211   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-04            638   873   518          0      175    189  
2020-11-05            658   897   529          0      179    203  
2020-11-06            681   922   542          0      188    294  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-04,3006,84,0,6059,2026,17808,2333,4645,712,0,...,3478,18771,625,58,23,3677,2416,474,2156,105
2020-11-05,3026,84,0,6087,2037,17860,2353,4656,716,0,...,3509,18907,632,58,23,3688,2431,482,2194,105
2020-11-06,3049,84,0,6109,2056,17931,2376,4671,716,0,...,3541,19046,649,58,23,3682,2439,489,2256,105


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-04          31      77       9   15     25      17     41      65   
2020-11-05          31      77       9   15     25      17     41      66   
2020-11-06          30      83       9   15     26      17     41      67   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-04           48       15  ...       3      4        6        1   
2020-11-05           48       15  ...       3      4        6        1   
2020-11-06           48       15  ...       3      4        6        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-04              4     2     3          0        7      1  
2020-11-05              4     2     3          0        7      1  
2020-11-06              4     2     3          0        7      1  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-04,0.002061,0.018079,0.009291,0.027495,0.020385,0.015625,0.008912,0.025049,0.000290,0.058385,...,0.023402,0.015332,0.003346,0.004082,0.000832,0.012309,0.0,0.000000,0.002221,0.002021
2020-11-05,0.002894,0.018879,0.010785,0.017839,0.024465,0.000000,0.009205,0.024838,0.000507,0.059281,...,0.021921,0.019723,0.003393,0.004718,0.003325,0.010376,0.0,0.000000,0.004312,0.002017
2020-11-06,0.000954,0.021522,0.010487,0.000000,0.009998,0.000000,0.009684,0.022197,0.000289,0.048779,...,0.020704,0.018435,0.001912,0.004089,0.004143,0.009776,0.0,0.001939,0.002922,0.003198


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-04,0.002591,0.007519,0.009596,0.0,0.017182,0.0,0.014601,0.024770,0.0,0.029362,...,0.010393,0.0,0.005226,0.004938,0.0,0.005988,0.0,0.000000,0.0,0.008130
2020-11-05,0.003876,0.013060,0.006003,0.0,0.010135,0.0,0.007565,0.019337,0.0,0.033415,...,0.007903,0.0,0.003466,0.006143,0.0,0.007937,0.0,0.000000,0.0,0.000000
2020-11-06,0.000000,0.011050,0.006464,0.0,0.003344,0.0,0.011292,0.020325,0.0,0.056782,...,0.007364,0.0,0.000000,0.002442,0.0,0.005906,0.0,0.001664,0.0,0.008065


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-04,0.000204,0.009152,0.010449,0.029501,0.006883,0.008547,0.008292,0.021042,0.000433,0.036935,...,0.006538,0.015768,0.002870,0.004378,0.000000,0.010054,0.0,0.0,0.003553,0.003148
2020-11-05,0.002270,0.010192,0.005927,0.033208,0.015951,0.000000,0.012273,0.022513,0.000354,0.046316,...,0.002051,0.015884,0.003892,0.004246,0.000000,0.009456,0.0,0.0,0.000506,0.001004
2020-11-06,0.000000,0.014107,0.006449,0.000000,0.051589,0.000000,0.011746,0.027388,0.000511,0.038497,...,0.006824,0.013859,0.001211,0.004487,0.000935,0.017605,0.0,0.0,0.002211,0.001003


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-04,0.009432,0.023613,NaN,0.003262,0.011291,0.005838,0.025525,0.007081,0.004248,0.0,...,0.012934,0.010006,0.017675,0.013411,0.002166,0.006265,0.013353,0.015315,0.024934,0.028931
2020-11-05,0.006983,0.017581,NaN,0.008518,0.013366,0.006240,0.028639,0.022382,0.008577,0.0,...,0.007298,0.010084,0.023106,0.015880,0.000000,0.007351,0.009598,0.021510,0.024270,0.024262
2020-11-06,0.007813,0.028887,NaN,0.007897,0.015934,0.006644,0.028618,0.013820,0.010950,0.0,...,0.005052,0.007975,0.024032,0.009987,0.001441,0.008376,0.015024,0.020341,0.024571,0.064638


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-04      0.006827  0.009435  0.004655  0.007865  0.012026  0.003044   
2020-11-05      0.008137  0.005098  0.000927  0.011148  0.015539  0.003035   
2020-11-06      0.005381  0.005213  0.009259  0.011025  0.013951  0.001513   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-04      0.004859  0.010099  0.014706  0.018564  ...  0.028226   
2020-11-05      0.002901  0.007598  0.008282  0.009721  ...  0.030065   
2020-11-06      0.006750  0.007541  0.005476  0.001203  ...  0.142132   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-04      0.081395  0.019518  0.015464   0.027375  0.012761  0.015686   
2020-11-05      0.021505  0.012387  0.015228   0.031348  0.027491  0.021236   
2020-11-06      0.089474  0.064516  0.055000   0.034954  0.027871  0.024575   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-04           -1.0  0.047904  0.044199  
2020-11-05            NaN  0.022857  0.074074  
2020-11-06            NaN  0.050279  0.448276  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-04,0.006361,0.0,NaN,0.006478,0.011483,0.003550,0.009520,0.002157,0.000000,NaN,...,0.006948,0.007460,0.008065,0.0,0.045455,0.003001,0.006667,0.006369,0.025690,0.129032
2020-11-05,0.006653,0.0,NaN,0.004621,0.005429,0.002920,0.008573,0.002368,0.005618,NaN,...,0.008913,0.007245,0.011200,0.0,0.000000,0.002992,0.006209,0.016878,0.017625,0.000000
2020-11-06,0.007601,0.0,NaN,0.003614,0.009327,0.003975,0.009775,0.003222,0.000000,NaN,...,0.009119,0.007352,0.026899,0.0,0.000000,-0.001627,0.003291,0.014523,0.028259,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                               \
Admin2           Autauga   Baldwin Barbour Bibb Blount Bullock Butler   
2020-11-04      0.000000  0.040541     0.0  0.0   0.00     0.0    0.0   
2020-11-05      0.000000  0.000000     0.0  0.0   0.00     0.0    0.0   
2020-11-06     -0.032258  0.077922     0.0  0.0   0.04     0.0    0.0   

Province_State                               ... Wyoming                     \
Admin2           Calhoun  Chambers Cherokee  ...    Park    Platte Sheridan   
2020-11-04      0.000000  0.021277      0.0  ...     0.0  0.333333      0.0   
2020-11-05      0.015385  0.000000      0.0  ...     0.0  0.000000      0.0   
2020-11-06      0.015152  0.000000      0.0  ...     0.0  0.000000      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-04          0.0        1.0   0.0   0.0        NaN      0.0    inf  
2020-11-05          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2020-11-06          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-04,0.002250,0.017175,0.007706,0.018655,0.022988,0.008441,0.009031,0.021101,0.000330,0.049309,...,0.021643,0.012176,0.003436,0.004103,0.003906,0.011905,1.275493e-42,0.000021,0.003713,0.001993
2020-11-05,0.002572,0.018027,0.009246,0.018247,0.023726,0.004221,0.009118,0.022970,0.000418,0.054295,...,0.021782,0.015950,0.003414,0.004410,0.003616,0.011141,6.377465e-43,0.000011,0.004013,0.002005
2020-11-06,0.001763,0.019774,0.009866,0.009124,0.016862,0.002110,0.009401,0.022583,0.000354,0.051537,...,0.021243,0.017192,0.002663,0.004249,0.003879,0.010458,3.188733e-43,0.000975,0.003467,0.002601


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-04,0.002302,0.010124,0.005782,0.000592,0.012940,3.919770e-62,0.013496,0.021582,0.000004,0.027713,...,0.008300,0.008013,0.004892,0.005089,4.311903e-21,0.009479,0.0,0.000313,0.000046,0.006267
2020-11-05,0.003089,0.011592,0.005892,0.000296,0.011538,1.959885e-62,0.010530,0.020460,0.000002,0.030564,...,0.008101,0.004006,0.004179,0.005616,2.155951e-21,0.008708,0.0,0.000157,0.000023,0.003133
2020-11-06,0.001545,0.011321,0.006178,0.000148,0.007441,9.799426e-63,0.010911,0.020392,0.000001,0.043673,...,0.007733,0.002003,0.002090,0.004029,1.077976e-21,0.007307,0.0,0.000910,0.000012,0.005599


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-04,0.000340,0.009102,0.006421,0.024993,0.023082,0.004938,0.010197,0.018631,0.000716,0.035608,...,0.009173,0.018830,0.003330,0.004480,0.001242,0.010358,7.473592e-45,0.000837,0.002694,0.002241
2020-11-05,0.001305,0.009647,0.006174,0.029100,0.019517,0.002469,0.011235,0.020572,0.000535,0.040962,...,0.005612,0.017357,0.003611,0.004363,0.000621,0.009907,3.736796e-45,0.000419,0.001600,0.001623
2020-11-06,0.000652,0.011877,0.006312,0.014550,0.035553,0.001234,0.011491,0.023980,0.000523,0.039729,...,0.006218,0.015608,0.002411,0.004425,0.000778,0.013756,1.868398e-45,0.000209,0.001906,0.001313


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-04,0.007701,0.023165,NaN,0.004469,0.009386,0.005388,0.024021,0.011922,0.005092,2.466227e-71,...,0.010289,0.008641,0.015715,0.011117,0.003156,0.006483,0.010384,0.015293,0.022980,0.030282
2020-11-05,0.007342,0.020373,NaN,0.006494,0.011376,0.005814,0.026330,0.017152,0.006834,1.233114e-71,...,0.008793,0.009363,0.019410,0.013498,0.001578,0.006917,0.009991,0.018401,0.023625,0.027272
2020-11-06,0.007577,0.024630,NaN,0.007195,0.013655,0.006229,0.027474,0.015486,0.008892,6.165569e-72,...,0.006923,0.008669,0.021721,0.011743,0.001509,0.007646,0.012508,0.019371,0.024098,0.045955


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-04      0.006343  0.006227  0.005215  0.007432  0.014751  0.002593   
2020-11-05      0.007240  0.005663  0.003071  0.009290  0.015145  0.002814   
2020-11-06      0.006311  0.005438  0.006165  0.010158  0.014548  0.002163   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-04      0.003649  0.008306  0.013767  0.016381  ...  0.026915   
2020-11-05      0.003275  0.007952  0.011025  0.013051  ...  0.028490   
2020-11-06      0.005013  0.007747  0.008250  0.007127  ...  0.085311   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-04      0.067097  0.021035  0.020246   0.029911  0.017546  0.019479   
2020-11-05      0.044301  0.016711  0.017737   0.030630  0.022519  0.020357   
2020-11-06      0.066887  0.040614  0.036369   0.032792  0.025195  0.022466   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-04           -1.0  0.036121  0.046027  
2020-11-05           -1.0  0.029489  0.060050  
2020-11-06           -1.0  0.039884  0.254163  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-04,0.005007,0.00424,NaN,0.005202,0.011426,0.002902,0.007399,0.001854,0.001433,NaN,...,0.010621,0.005708,0.007968,7.142256e-30,0.034632,0.002367,0.006398,0.009914,0.021381,0.069727
2020-11-05,0.005830,0.00212,NaN,0.004912,0.008428,0.002911,0.007986,0.002111,0.003525,NaN,...,0.009767,0.006477,0.009584,3.571128e-30,0.017316,0.002679,0.006303,0.013396,0.019503,0.034863
2020-11-06,0.006716,0.00106,NaN,0.004263,0.008878,0.003443,0.008880,0.002666,0.001763,NaN,...,0.009443,0.006914,0.018241,1.785564e-30,0.008658,0.000526,0.004797,0.013959,0.023881,0.017432


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-11-04      0.000010  0.031287  1.064369e-09  0.000141  0.000002   
2020-11-05      0.000005  0.015644  5.321843e-10  0.000070  0.000001   
2020-11-06     -0.016127  0.046783  2.660922e-10  0.000035  0.020001   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2020-11-04      2.986375e-08  0.000049  0.001311  0.010777  0.001116  ...   
2020-11-05      1.493188e-08  0.000024  0.008348  0.005389  0.000558  ...   
2020-11-06      7.465938e-09  0.000012  0.011750  0.002694  0.000279  ...   

Province_State   Wyoming                                                  \
Admin2              Park    Platte  Sheridan Sublette Sweetwater   Teton   
2020-11-04      0.000008  0.292643  0.081250      0.0      0.500  0.2500   
2020-11-05      0.000004  0.146322  0.040625      0.0      0.250  0.1250   
2020-11-06      0.000002  0.073161  0.020313      0.0      0.125  0.0625   

Province_State                                            
Admin2             Uinta Unassigned      Washakie Weston  
2020-11-04      0.000061  -0.516251  1.986821e-08    NaN  
2020-11-05      0.000031  -0.516251  9.934107e-09    0.0  
2020-11-06      0.000015  -0.516251  4.967054e-09    0.0  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201106,AK,18699,NaN,762602,NaN,totalTestsViral,781301,106.0,NaN,...,781301,0,0,c5e7641b5775740f5912af833d4d6b5c452c3383,0,0,0,0,0,NaN
1,20201106,AL,200714,30268.0,1217002,NaN,totalTestsViral,1387448,1022.0,21294.0,...,1417716,23,267,3c28ba3b012f38332635f3629ab4fa30bddaf663,0,0,0,0,0,NaN
2,20201106,AR,119230,10492.0,1298264,NaN,totalTestsViral,1407002,633.0,7401.0,...,1417494,19,75,857a88ae2ba0c49b94fd2cff5909d3578d1c7d2a,0,0,0,0,0,NaN
3,20201106,AS,0,NaN,1768,NaN,totalTestsViral,1768,NaN,NaN,...,1768,0,0,7113b8d9b4c8875c13e403e62e224421d71d7d96,0,0,0,0,0,NaN
4,20201106,AZ,254764,6434.0,1593981,NaN,totalTestsPeopleViral,1842311,1082.0,22018.0,...,1848745,22,187,f26b37d31cec9fcaa2c54e30e98dba367ba5686c,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201106,AK,18699.0,NaN,762602.0,NaN,totalTestsViral,781301.0,106.0,NaN,...,781301,0,0,c5e7641b5775740f5912af833d4d6b5c452c3383,0,0,0,0,0,NaN
1,20201106,AL,200714.0,30268.0,1217002.0,NaN,totalTestsViral,1387448.0,1022.0,21294.0,...,1417716,23,267,3c28ba3b012f38332635f3629ab4fa30bddaf663,0,0,0,0,0,NaN
2,20201106,AR,119230.0,10492.0,1298264.0,NaN,totalTestsViral,1407002.0,633.0,7401.0,...,1417494,19,75,857a88ae2ba0c49b94fd2cff5909d3578d1c7d2a,0,0,0,0,0,NaN
3,20201106,AS,0.0,NaN,1768.0,NaN,totalTestsViral,1768.0,NaN,NaN,...,1768,0,0,7113b8d9b4c8875c13e403e62e224421d71d7d96,0,0,0,0,0,NaN
4,20201106,AZ,254764.0,6434.0,1593981.0,NaN,totalTestsPeopleViral,1842311.0,1082.0,22018.0,...,1848745,22,187,f26b37d31cec9fcaa2c54e30e98dba367ba5686c,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-06,AK,18699,NaN,762602,NaN,totalTestsViral,781301,106.0,NaN,NaN,...,781301,0,0,c5e7641b5775740f5912af833d4d6b5c452c3383,0,0,0,0,0,NaN
2020-11-06,AL,200714,30268.0,1217002,NaN,totalTestsViral,1387448,1022.0,21294.0,NaN,...,1417716,23,267,3c28ba3b012f38332635f3629ab4fa30bddaf663,0,0,0,0,0,NaN
2020-11-06,AR,119230,10492.0,1298264,NaN,totalTestsViral,1407002,633.0,7401.0,235.0,...,1417494,19,75,857a88ae2ba0c49b94fd2cff5909d3578d1c7d2a,0,0,0,0,0,NaN
2020-11-06,AS,0,NaN,1768,NaN,totalTestsViral,1768,NaN,NaN,NaN,...,1768,0,0,7113b8d9b4c8875c13e403e62e224421d71d7d96,0,0,0,0,0,NaN
2020-11-06,AZ,254764,6434.0,1593981,NaN,totalTestsPeopleViral,1842311,1082.0,22018.0,250.0,...,1848745,22,187,f26b37d31cec9fcaa2c54e30e98dba367ba5686c,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-06,AK,18699.0,NaN,762602.0,NaN,totalTestsViral,781301.0,106.0,NaN,NaN,...,781301,0,0,c5e7641b5775740f5912af833d4d6b5c452c3383,0,0,0,0,0,NaN
2020-11-06,AL,200714.0,30268.0,1217002.0,NaN,totalTestsViral,1387448.0,1022.0,21294.0,NaN,...,1417716,23,267,3c28ba3b012f38332635f3629ab4fa30bddaf663,0,0,0,0,0,NaN
2020-11-06,AR,119230.0,10492.0,1298264.0,NaN,totalTestsViral,1407002.0,633.0,7401.0,235.0,...,1417494,19,75,857a88ae2ba0c49b94fd2cff5909d3578d1c7d2a,0,0,0,0,0,NaN
2020-11-06,AS,0.0,NaN,1768.0,NaN,totalTestsViral,1768.0,NaN,NaN,NaN,...,1768,0,0,7113b8d9b4c8875c13e403e62e224421d71d7d96,0,0,0,0,0,NaN
2020-11-06,AZ,254764.0,6434.0,1593981.0,NaN,totalTestsPeopleViral,1842311.0,1082.0,22018.0,250.0,...,1848745,22,187,f26b37d31cec9fcaa2c54e30e98dba367ba5686c,0,0,0,0,0,NaN
